In [36]:
import pandas as pd
import os
import datetime

In [34]:
import re

with open("../Dataset/Subtitles/NDT_India.srt", 'r') as h:
    sub = h.readlines()

re_pattern = r'[0-9]{2}:[0-9]{2}:[0-9]{2},[0-9]{3} -->'
regex = re.compile(re_pattern)
# Get start times
start_end_times = list(filter(regex.search, sub))
start_times = [time.split(' --> ')[0] for time in start_end_times]
end_times = [time.split(' --> ')[1][:-1] for time in start_end_times]

# Get lines
lines = []
for idx in range(len(sub)):
    if (idx + 2) % 4 == 0:
        lines.append(sub[idx][:-1])

In [68]:
df = pd.DataFrame({"start_time": start_times, "end_time": end_times, "line": lines})
df["start_time"] = pd.to_datetime(df["start_time"], format='%H:%M:%S,%f')
df["start_time"] = df["start_time"].apply(lambda x: x.hour*3600 + x.minute * 60 + x.second + x.microsecond/1000000)
df["end_time"] = pd.to_datetime(df["end_time"], format='%H:%M:%S,%f')
df["end_time"] = df["end_time"].apply(lambda x: x.hour*3600 + x.minute * 60 + x.second + x.microsecond/1000000)
df

,start_time,end_time,line
0,0.120,2.540,we choose to go to the Moon
1,2.540,6.560,we choose to go to the Moon
2,10.920,13.500,we choose to go to the moon in this
3,13.500,15.839,decade and do the other things not
4,15.839,18.359,because they are easy but because they
...,...,...,...
76,179.040,180.959,watching don't miss the Today show every
77,180.959,184.140,weekday at 11 A.M Eastern 8 Pacific on
78,184.140,186.360,our streaming Channel today all day to
79,186.360,190.680,watch head to today.com all day or click


In [67]:
df

,start_time,end_time,line
0,0.120,1900-01-01 00:00:02.540,we choose to go to the Moon
1,2.540,1900-01-01 00:00:06.560,we choose to go to the Moon
2,10.920,1900-01-01 00:00:13.500,we choose to go to the moon in this
3,13.500,1900-01-01 00:00:15.839,decade and do the other things not
4,15.839,1900-01-01 00:00:18.359,because they are easy but because they
...,...,...,...
76,179.040,1900-01-01 00:03:00.959,watching don't miss the Today show every
77,180.959,1900-01-01 00:03:04.140,weekday at 11 A.M Eastern 8 Pacific on
78,184.140,1900-01-01 00:03:06.360,our streaming Channel today all day to
79,186.360,1900-01-01 00:03:10.680,watch head to today.com all day or click
